In [27]:
# 加更多的层总是改进精度吗?
#  - Non-nested function classes: 加入的新模有更深的层, 但是不包含子模型的话, 会偏离答案
#  - Nested function classes: 加入的新模型, 每次都包含子模型的话, 新的模型会比原来更大, 但是不偏差

# 残差块
# 串联一个层改变函数, 我们希望能够扩大函数类
# 残差块加入快速通道来得到 f(x) = g(x)+x的结构

# 原来的: x -> (Weight layer -> Activation function -> Weight layer) = f(x) -> Activation function
# 现在的: x -> (Weight layer -> Activation function -> Weight layer) = f(x) = f(x) + x -> Activation function
#          -> (捷径) -> Activation function (same)
# 就是复杂模型包含小模型

# ResNet细节
# x -> 3x3 Conv -> Batch Norm -> ReLU -> 3x3 Conv -> Batch Norm -> + -> ReLu
# x -> 3x3 Conv -> Batch Norm -> ReLU -> 3x3 Conv -> Batch Norm -> + -> ReLu
# x -> 1x1 Conv -> + (1x1的卷积层变换通道, 变到合适的范围)
# 可以在在任何的地方

# ResNet块
#  - 高宽减半ResNet块(步幅2): 在第一卷积层, 步幅为2 
#  - 后接多个高宽不变的ResNet块

# ResNet架构
#  - 类似VGG和GoogleNet的架构但是替换成了ResNet块

# 总结
#  - 残差块使得很深的网络更加容易训练
#  - 残差网络对随后的深层神经网络产生了深远的影响

In [28]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

class Residual(nn.Module):
    def __init__(self, input_channels, num_channels, use_1x1conv=False,
                 strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels, kernel_size=3,
                               padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3,
                               padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
        self.relu = nn.ReLU(inplace=True) # 进行原地计算, 省内存
    
    def forward(self, X):
        # 根据上面的图来制作的
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X # 把 X + Y
        return F.relu(Y)

In [29]:
# 输入和输出形状一样
blk = Residual(3, 3)
X = torch.rand(4, 3, 6, 6)
Y = blk(X)
Y.shape

torch.Size([4, 3, 6, 6])

In [30]:
# 增加通道数, 同时减半输出的高宽
blk = Residual(3, 6, use_1x1conv=True, strides=2)
blk(X).shape

torch.Size([4, 6, 3, 3])

In [32]:
# ResNet模型
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
def resnet_block(input_channels, num_channels, num_residuals, first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block: # 如果不是第一个残差块, 且是目前块的第一残差单元, 那么做步长为2, 1x1的下采样
            blk.append(Residual(input_channels, num_channels, use_1x1conv=True,
                                strides=2))
        else:
            # 第一个残差块用于保留原始信息
            blk.append(Residual(num_channels, num_channels))
    return blk

b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

net = nn.Sequential(b1, b2, b3, b4, b5, nn.AdaptiveAvgPool2d((1, 1)),
                    nn.Flatten(), nn.Linear(512, 10))

In [33]:
# 观察一下网络结构
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])


In [ ]:
lr, num_epochs, batch_size = 0.05, 10, 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())